In [9]:
from keras.datasets import mnist  #Used to load the mnist dataset
import matplotlib.pyplot as plt   #Used to show graphs (All graphs are commented out)
import numpy as np  
import time                       #Used to calculate elapsed time
np.set_printoptions(suppress=True)  #Disables scientific notation when printing


In [10]:
#Data loading using keras mnist.load_data() function
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#Reduces the number of dimensions from 3 to 2
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1]*X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1]*X_test.shape[2])
y_train = np.array(y_train, dtype=np.int8)
y_test = np.array(y_test, dtype=np.int8)

In [11]:
#Encoding our y data to 2 classes , for even and odd numbers.
def encode(A):
  xE= []
  for x in range(len(A)):
    if (A[x]%2==0):
      value = 1
    else:
      value=-1
    xE.append(value)
  return np.array(xE)

#Normalizing our data by dividing them all by 255
def normalize(X_array):
  X_array=X_array/255
  X_array=np.round(X_array,5)
  return X_array

In [12]:
#Applying the above functions to our data

y_train = encode(y_train)
y_test = encode(y_test)
X_train = normalize(X_train)
X_test = normalize(X_test)


In [13]:
#Τaking smaller samples of "size" data to use
index = np.random.choice(X_train.shape[0], size=10000, replace=False)
index2 = np.random.choice(X_test.shape[0], size=1000, replace=False)
X_train_sm=X_train[index]
y_train_sm=y_train[index]
X_test_sm=X_test[index2]
y_test_sm=y_test[index2]

In [14]:
#Object for our svm. Initializes weights and bias to 0
class Layer:
  def __init__(self,X):
    self.weights = np.zeros(len(X[0]))
    self.b = 0   #Bias of each layer
  


#Function used to test our model after it's trained.
def classify(svm,X,y):
  correct = 0
  incorrect = 0
  for x in range(len(X)):
    temp = np.sign(np.dot(X[x],svm.weights) - svm.b) #Tests with : xT*w - b
    if (temp == y[x]):
      correct+=1
    else:
      incorrect+=1

  print("\nNo of correct classifications : ",correct)
  print("No of incorrect classifications : ",incorrect)
  print("Accuracy percentage : ",correct*100/len(X))


In [ ]:
#Soft Margin without Quadratic Programming
start = time.time()

#Using smaller training sets to avoid overfitting
X = X_train_sm
y = y_train_sm
svmT = Layer(X)

#Learning rate and lamda hyperparameters
lamda = 0.1 #Lamda parameter
lr = 0.001  #Learning rate

#Number of epochs for backpropagation. 1 is enough but can be increased for smaller learning rate.
epochs = 1
loss = np.zeros(epochs)
#For each epoch -> for each sample of data, calculate output and update weights and bias based on if it's on the correct side.
for e in range(epochs):
  tempLoss = 0
  for i in range(len(X)):
    output = y[i] * (np.dot(X[i],svmT.weights) - svmT.b)

    #Loss is increased only if sample is classified incorrectly
    if (output < 1):
      tempLoss += 1 - output
      svmT.weights -= lr * (2 * lamda * svmT.weights - np.dot(X[i],y[i]))
    else:
      svmT.weights -= lr * (2 * lamda * svmT.weights)
      svmT.b -= lr * y[i]

  #Loss for each epoch. Can be plotted for epcohs>1    
  loss[e] = tempLoss/784 - lamda *np.dot(svmT.weights,svmT.weights)

#Classify test data 
X = X_test #X_test_sm + y_test_sm can be used for smaller test set
y = y_test 
classify(svmT,X,y)

end = time.time()
print ("\nTime to train and test model: ",float("{:.2f}".format((end - start)/60))," minutes")



print("Loss: ",loss[-1])

In [16]:
start = time.time()
#Soft margin with Quadratic Programming and no kernel
X = X_train_sm
y = y_train_sm
svmQuad = Layer(X)

#Hyperparameter C
C = 0.1 

#Using quadratic solver CVXOPT
#Converting our problem to one solvable by our quad solver
#Calculating matrices for our quadratic solver so that our problem is:
#Minimize 1/2xT Px + qTx #subject to Gx<=h and Ax=b


Q = np.dot(X,X.T)
Q2 = np.outer(y,y)

#Matrix P = Σyk*yl(xk.xl)
P = Q*Q2

q =  - np.ones((len(X),1))
b = np.zeros(1)
#A matrix is calculated directly as a cvxopt matrix
G_A = -np.eye(len(X)) 
G = np.vstack((G_A, np.identity(len(X))))
h = np.hstack((np.zeros(len(X)), np.ones(len(X)) * C))

#Converting our numpy arrays to cvxopt matrices
from cvxopt import matrix 
from cvxopt import solvers 
P = matrix(P)
q = matrix(q)
G = matrix(G)
h = matrix(h)
A = matrix(y, (1,len(X)),'d')
b = matrix(b)

quadSolution = solvers.qp(P, q, G, h, A, b)

#Taking the alphas from the solved function
alphas = np.array(quadSolution['x'])

#Taking the indexes of the alphas where a>0 (a>0.00001)
index = []
alphasBool = (alphas > 1e-5)
for i in range(len(alphasBool)):
  if alphasBool[i]==True:
    index.append(i) 

#Taking only the samples where a>0
aSV = alphas[index]
XSV = X[index]
ySV = y[index]
ySV = ySV.reshape(-1,1)

#Calculating weights and bias
svmQuad.weights =np.dot(XSV.T,ySV*aSV)

#Bias is taken as a mean from every support vector
for i in range(len(XSV)):                           
  svmQuad.b += np.dot(svmQuad.weights.T,XSV[i]) - ySV[i]
svmQuad.b /= len(XSV)

#Classifying test data
X = X_test
y = y_test
classify(svmQuad,X,y)

X = X_train
y = y_train
classify(svmQuad,X,y)
end = time.time()
print ("\nTime to train and test model: ",float("{:.2f}".format((end - start)/60))," minutes")

     pcost       dcost       gap    pres   dres
 0: -1.9448e+03 -2.4075e+03  1e+05  3e+01  1e-12
 1: -3.5447e+02 -2.2639e+03  8e+03  2e+00  1e-12
 2: -2.4282e+02 -1.4231e+03  2e+03  4e-01  4e-13
 3: -2.1203e+02 -7.8746e+02  9e+02  2e-01  2e-13
 4: -2.0507e+02 -3.8032e+02  2e+02  4e-02  2e-13
 5: -2.1807e+02 -2.8381e+02  8e+01  1e-02  2e-13
 6: -2.2377e+02 -2.6283e+02  4e+01  5e-03  2e-13
 7: -2.2754e+02 -2.5141e+02  3e+01  2e-03  2e-13
 8: -2.3031e+02 -2.4161e+02  1e+01  2e-15  3e-13
 9: -2.3229e+02 -2.3885e+02  7e+00  7e-15  2e-13
10: -2.3395e+02 -2.3652e+02  3e+00  2e-15  3e-13
11: -2.3472e+02 -2.3556e+02  8e-01  7e-16  3e-13
12: -2.3500e+02 -2.3520e+02  2e-01  5e-15  3e-13
13: -2.3508e+02 -2.3511e+02  2e-02  9e-16  3e-13
14: -2.3510e+02 -2.3510e+02  2e-03  2e-15  3e-13
15: -2.3510e+02 -2.3510e+02  7e-05  7e-15  3e-13
Optimal solution found.

No of correct classifications :  8966
No of incorrect classifications :  1034
Accuracy percentage :  89.66

No of correct classifications :  53

In [8]:
#Using sklearn to create Polynomial and RBF Kernel SVMs to compare with no kernel soft margin
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
start = time.time()

#SVM With poly kernel and degree 3
clf = SVC(C=100,kernel='rbf',gamma=0.01)
clf.fit(X_test_sm,y_test_sm)

accuracyTrain = accuracy_score(y_train,clf.predict(X_train))
accuracyTest = accuracy_score(y_test,clf.predict(X_test))

print("Accuracy on train dataset: ",accuracyTrain*100,"%")
print("Accuracy on test dataset: ",accuracyTest*100,"%")

end = time.time()
print ("\nTime to train and test model: ",float("{:.2f}".format((end - start)/60))," minutes")

Accuracy on train dataset:  94.62 %
Accuracy on test dataset:  95.78999999999999 %

Time to train and test model:  0.29  minutes


In [ ]:
#Using GridSearchCV to apply multiple parameter combinations and find the best
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score

start = time.time()

param_grid = [
            {"C": [0.1, 1, 10,100],"degree":[2,3,4] ,"kernel" : ["poly"]},
            {"C": [0.1, 1, 10, 100], "gamma": [0.0001,0.001,0.1,1], "kernel": ["rbf"]}]

clf = GridSearchCV(SVC(),param_grid)
clf.fit(X_train_sm, y_train_sm)
end = time.time()

In [112]:
#Data regarding the Parameter grid results from GridSearchCV  - - clf.best_params_ returns the best parameter combination from our grid
scores = clf.cv_results_
print(scores)
print(clf.best_params_)

In [ ]:
#SVM With poly kernel and degree 3
clf = SVC(C=10,kernel='poly',degree=3)
clf.fit(X_test_sm,y_test_sm)

accuracyTrain = accuracy_score(y_train,clf.predict(X_train))
accuracyTest = accuracy_score(y_test,clf.predict(X_test))

print("Accuracy on train dataset: ",accuracyTrain*100,"%")
print("Accuracy on test dataset: ",accuracyTest*100,"%")
